In [1]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import matplotlib.pyplot as plt

In [2]:
tickers = { 'Govenment' : ['XGB.TO', 'VGV.TO'],
           'Candian Stocks' : ['BESG.TO', 'XCG.TO', 'XCV.TO', 'XCSR.TO', 'XESG.TO', 'GGRO.TO' ],
           'US Stocks' : ['ESGV', 'VUG', 'VV', 'MGC', 'MGC', 'MGV', 'VOO', 'VOOG', 'VOOV', 'VTV', 'VIOO', 'VIOG', 'VIOV', 'VB', 'VBK', 'VBR', 'FCMO.TO', 'ESG.TO', 'XUS.TO', 'XUSR.TO', 'XSUS.TO', 'XMTM.TO'],
           'US reits' : ['VNQ', 'XRE.TO'],
           'International' : ['VSGX', 'VEA', 'VWO'],
           'Developed & Emerging Markets' : ['VWOB', 'VGT', 'VDE', 'XEM.TO', 'VIU.TO', 'VEE.TO', 'VMO.TO', 'VVL.TO', 'VGRO.TO', 'EMB', 'ESGE'] 
           }

In [3]:
stock_list = []
for i in tickers:
    for n in tickers[i]: # iterate for every stock indices
        # Retrieve data from Yahoo! Finance
        tickerData = yf.Ticker(n)
        tickerDf1 = tickerData.history(period='1d', start='2010-1-1', end='2020-9-30')
        # Save historical data 
        tickerDf1['ticker'] = n # don't forget to specify the index
        tickerDf1['asset class'] = i
        tickerDf1['Returns'] = tickerDf1['Close']/tickerDf1['Close'].shift(1)
        stock_list.append(tickerDf1)
# Concatenate all data
msi = pd.concat(stock_list, axis = 0)
msi.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,asset class,Returns
Date,,,,,,,,,,
2010-01-04,14.925273,14.947418,14.925273,14.932654,51800.0,0.0,0.0,XGB.TO,Govenment,NaN
2010-01-05,14.940039,14.991709,14.940039,14.954803,49900.0,0.0,0.0,XGB.TO,Govenment,1.001483
2010-01-06,14.984322,14.991703,14.925270,14.925270,27100.0,0.0,0.0,XGB.TO,Govenment,0.998025
2010-01-07,14.925272,14.969562,14.925272,14.969562,15300.0,0.0,0.0,XGB.TO,Govenment,1.002968
2010-01-08,14.969562,14.991706,14.962181,14.976944,34100.0,0.0,0.0,XGB.TO,Govenment,1.000493


In [4]:
# Transform the data to be ticker column-wise
Gross_rets = msi.groupby(['Date', 'ticker'])['Returns'].first().unstack()
rets = Gross_rets - 1
# Fill null values with the values on the row before

rets.head()

ticker,BESG.TO,EMB,ESG.TO,ESGE,ESGV,FCMO.TO,GGRO.TO,MGC,MGV,VB,...,XCSR.TO,XCV.TO,XEM.TO,XESG.TO,XGB.TO,XMTM.TO,XRE.TO,XSUS.TO,XUS.TO,XUSR.TO
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,NaN,0.006083,NaN,NaN,NaN,NaN,NaN,0.003071,0.002799,0.003071,...,NaN,-0.000534,0.006367,NaN,0.001483,NaN,0.000864,NaN,NaN,NaN
2010-01-06,NaN,0.001365,NaN,NaN,NaN,NaN,NaN,0.000510,0.002512,0.000681,...,NaN,0.002135,-0.002372,NaN,-0.001975,NaN,-0.004314,NaN,NaN,NaN
2010-01-07,NaN,-0.005161,NaN,NaN,NaN,NaN,NaN,0.003824,0.006126,0.004761,...,NaN,-0.006926,-0.003567,NaN,0.002968,NaN,0.007799,NaN,NaN,NaN
2010-01-08,NaN,0.001174,NaN,NaN,NaN,NaN,NaN,0.002032,-0.001107,0.007445,...,NaN,0.001610,0.001193,NaN,0.000493,NaN,0.006878,NaN,NaN,NaN


In [ ]:

fig, axes = plt.subplots(3,2, figsize=(12, 8),sharex=True)
pagoda = ["#965757", "#D67469", "#4E5A44", "#A1B482", '#EFE482', "#99BFCF"] # for coloring
for i, k in enumerate(tickers.keys()):
# Iterate for each region
    ax = axes[int(i/2), int(i%2)]
    for j,t in enumerate(tickers[k]):
        print(t)
        # Iterate and plot for each stock index in this region
        ax.plot(rets.index, rets[t], marker='', linewidth=0.5, color = pagoda[j])
        ax.set_title(k, fontweight='bold')
fig.text(0.5,0, "Year", ha="center", va="center", fontweight ="bold")
fig.text(0,0.5, "Price Change/Return (%)", ha="center", va="center", rotation=90, fontweight ="bold")
fig.suptitle("Price Change/Return for Major Stock Indices based on 2010", fontweight ="bold",y=1.05, fontsize=14)
fig.tight_layout()

In [ ]:
xrets = rets.values - rets['XGB.TO'].values.reshape(2752,1)
xrets_df = pd.DataFrame(xrets, index = rets.index, columns = rets.columns)
xrets_df = xrets_df.drop('XGB.TO', axis = 1)

xrets_df.head()

In [ ]:
# T-Statistic test 
from scipy.stats import t

l_etf = []
T = len(xrets)
tcrit = t.pdf(1 - 0.01, T-1)

null = 0
se = np.std(xrets_df) / np.sqrt(2752)
tstat = (np.mean(xrets_df) - null) / se
p_val = 2*(1-t.cdf(abs(tstat), T-1))
for i, n in enumerate(tstat):
    if abs(n) > tcrit:
        l_etf.append(tstat.index[i])
print(p_val)
